In [181]:
from keras.models import load_model
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
import pymongo 

from plotly.offline import iplot, init_notebook_mode, plot
import plotly.graph_objs as go

init_notebook_mode(connected=True)

In [2]:

# loading model
model = load_model('./models/keras_model.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [13]:
# preparing data

#myclient = pymongo.MongoClient("mongodb://localhost:27017/")

myclient = pymongo.MongoClient('mongodb://192.168.5.103/crypto_db')
mydb = myclient["crypto_db"]

In [17]:
iota_data = pd.DataFrame(list(mydb['IOTA'].find()))

iota = iota_data[iota_data['Close Time'] > '2019-03-01 00:00:00.000'][['Close']]
iota['Close'] = iota['Close'].astype(float)

scaler = MinMaxScaler(feature_range=(0,1))
iota['Close'] = scaler.fit_transform(iota['Close'].values.reshape(-1, 1))

for i in range(-1, -31, -1):
    column_name = f'Close_{i*-1}'
    iota[column_name] = iota['Close'].shift(i)
    
iota.dropna(inplace=True)

iota.head()

,Close,Close_1,Close_2,Close_3,Close_4,Close_5,Close_6,Close_7,Close_8,Close_9,...,Close_21,Close_22,Close_23,Close_24,Close_25,Close_26,Close_27,Close_28,Close_29,Close_30
392498,0.208129,0.208129,0.207682,0.209915,0.207235,0.208129,0.209022,0.209022,0.209022,0.209915,...,0.206789,0.206789,0.207235,0.207682,0.207682,0.207682,0.209468,0.209468,0.209022,0.209022
392499,0.208129,0.207682,0.209915,0.207235,0.208129,0.209022,0.209022,0.209022,0.209915,0.207682,...,0.206789,0.207235,0.207682,0.207682,0.207682,0.209468,0.209468,0.209022,0.209022,0.209468
392500,0.207682,0.209915,0.207235,0.208129,0.209022,0.209022,0.209022,0.209915,0.207682,0.208575,...,0.207235,0.207682,0.207682,0.207682,0.209468,0.209468,0.209022,0.209022,0.209468,0.211255
392501,0.209915,0.207235,0.208129,0.209022,0.209022,0.209022,0.209915,0.207682,0.208575,0.208575,...,0.207682,0.207682,0.207682,0.209468,0.209468,0.209022,0.209022,0.209468,0.211255,0.211255
392502,0.207235,0.208129,0.209022,0.209022,0.209022,0.209915,0.207682,0.208575,0.208575,0.208575,...,0.207682,0.207682,0.209468,0.209468,0.209022,0.209022,0.209468,0.211255,0.211255,0.207682


In [157]:
def bulk_predictions(num_predictions, coin):
    data = pd.DataFrame(list(mydb[coin].find()))
    data = data.tail(31)[['Close']]
    data['Close'] = data['Close'].astype(float)
    
    scaler = MinMaxScaler(feature_range=(0,1))
    data['Close'] = scaler.fit_transform(data['Close'].values.reshape(-1, 1))
    
    for i in range(-1, -31, -1):
        column_name = f'Close_{i*-1}'
        data[column_name] = data['Close'].shift(i)

    data.dropna(inplace=True)
    
    predictions = []
    og = data.iloc[-1, :-1].values #.tolist()

    for i in range(num_predictions):
        ex = og

        ex = np.append(np.array(predictions), ex)
        ex = ex[0:30]

        pred = model.predict(np.array(ex).reshape(1, 1, 30))
        predictions = np.append(pred[0], np.array(predictions))
        
    return scaler.inverse_transform(np.array(predictions).reshape(-1, 1))

In [195]:
start = datetime.now()
preds = bulk_predictions(num_predictions=120, coin='IOTA')
print(f'That took: {datetime.now() - start}')

That took: 0:00:22.912618


In [196]:
ts = []
max_ = iota_data['Close Time'].max()
for i in range(120):
    ts.append(
        max_ + timedelta(seconds=60*i)
    )


In [197]:
forecast = pd.DataFrame()

In [198]:
forecast['Close Time'] = ts

In [199]:
forecast['Forecast'] = preds

In [200]:
prev_40 = iota_data.tail(40)

actual = go.Scatter(
    x = prev_40['Close Time'],
    y = prev_40['Close'],
    name = 'Actual'
)

fore = go.Scatter(
    x = forecast['Close Time'],
    y = forecast['Forecast'],
    name = 'Forecast'
)

layout = go.Layout(
    title = 'IOTA-USDT Forecast'
)

fig = go.Figure([actual, fore], layout)
iplot(fig)